In [1]:
import os
import pandas as pd
import numpy as np
import sys
sys.path.append('/gpfs2/projects/project-bus_capstone_2016/workspace/mu529/Bus-Capstone')
import ttools
os.chdir('../..')
os.chdir('share')

In [2]:
avl_data = pd.read_csv('newdata_parsed.csv')

In [3]:
oneday = avl_data.query('TripDate=="2016-06-13"')

In [6]:
oneday.query('Line == "MTA NYCT_B26"').to_csv('2016-06-13_lineB26_sample.csv')

In [9]:
oneday = oneday.sort(columns=['vehicleID','RecordedAtTime'])

In [10]:
oneday.drop_duplicates(subset=['vehicleID','RecordedAtTime','Latitude','Longitude'],inplace=True)

In [14]:
oneday['RecordedAtTime_parsed'] = oneday.RecordedAtTime.apply(ttools.parseActualTime,tdate='2016-06-13')

That finishes the parsing of the vehicle ping times.
Next is to look at the time deltas between the Siri API responses.

In [39]:
responsetimes = oneday.ResponseTimeStamp.unique()

In [40]:
responsetimes.sort()

In [41]:
responsetimes = pd.Series(data=responsetimes,index=responsetimes)

In [42]:
responsetimes = responsetimes.apply(ttools.parseActualTime,tdate='2016-06-13')

In [43]:
responsetimediffs = pd.DataFrame(responsetimes.diff(),columns=['TimeSinceLastSiriResponse'])

In [44]:
responsetimediffs.head()

,TimeSinceLastSiriResponse
2016-06-13T02:10:40.022-04:00,NaT
2016-06-13T02:11:15.880-04:00,00:00:35
2016-06-13T02:11:51.152-04:00,00:00:36
2016-06-13T02:12:25.817-04:00,00:00:34
2016-06-13T02:13:00.099-04:00,00:00:35


In [48]:
oneday = oneday.merge(responsetimediffs,how='left',left_on='ResponseTimeStamp',right_index=True)

,vehicleID,Line,RecordedAtTime,Latitude,Longitude,Trip,TripDate,ResponseTimeStamp,RecordedAtTime_parsed,TimeSinceLastSiriResponse
663836,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:05:40.000-04:00,40.786901,-73.950176,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:06:00.048-04:00,13:05:40,00:01:28
666177,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:07:14.000-04:00,40.785463,-73.951226,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:07:33.389-04:00,13:07:14,00:00:03
1156394,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:08:28.000-04:00,40.783989,-73.952304,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:08:53.334-04:00,13:08:28,00:00:22
3826081,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:09:32.000-04:00,40.780415,-73.954914,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:10:00.135-04:00,13:09:32,00:00:04
4435928,MTA NYCT_1001,MTA NYCT_M101,2016-06-13T13:10:04.000-04:00,40.779581,-73.955523,MTA NYCT_OH_B6-Weekday-SDon-077600_M101_100,2016-06-13,2016-06-13T13:10:09.167-04:00,13:10:04,00:00:09
